In [29]:
import numpy as np
import pandas as pd
import cv2
from scipy.spatial.transform import Rotation as R

ROBOT_CSV = '/home/lgx/Project/hand_eye_calib/src/calibrate/raw_data/robot_poses.csv'
VISION_CSV = '/home/lgx/Project/hand_eye_calib/src/calibrate/raw_data/vision_poses.csv'
HAND_EYE_FILE = '/home/lgx/Project/hand_eye_calib/src/calibrate/result_hand_eye.txt'

def make_T(R_mat, t_vec):
    T = np.eye(4)
    T[:3, :3] = R_mat
    T[:3, 3] = t_vec.reshape(3)
    return T

def main():
    print("=== 手眼标定结果验证（眼在手外，棋盘在末端） ===")

    # 1. 读取数据
    df_robot = pd.read_csv(ROBOT_CSV)
    df_vision = pd.read_csv(VISION_CSV)
    T_base_cam = np.loadtxt(HAND_EYE_FILE)

    print(f"机器人数据: {len(df_robot)}")
    print(f"视觉数据: {len(df_vision)}")

    T_ET_list = []

    # 2. 遍历所有有效帧
    for i, row_vis in df_vision.iterrows():
        idx = int(row_vis['img_index'])
        if idx >= len(df_robot):
            continue

        # ---------- Base -> End ----------
        row_r = df_robot.iloc[idx]
        t_be = np.array([row_r['x'], row_r['y'], row_r['z']])
        q_be = [row_r['qx'], row_r['qy'], row_r['qz'], row_r['qw']]
        R_be = R.from_quat(q_be).as_matrix()
        T_be = make_T(R_be, t_be)

        # ---------- Camera -> Target ----------
        t_ct = np.array([
            row_vis['cam_tx'],
            row_vis['cam_ty'],
            row_vis['cam_tz']
        ])
        rvec_ct = np.array([
            row_vis['cam_rx'],
            row_vis['cam_ry'],
            row_vis['cam_rz']
        ]).reshape(3, 1)
        R_ct, _ = cv2.Rodrigues(rvec_ct)
        T_ct = make_T(R_ct, t_ct)

        # ---------- End -> Target ----------
        T_et = np.linalg.inv(T_be) @ T_base_cam @ T_ct
        T_ET_list.append(T_et)

    if len(T_ET_list) < 3:
        print("❌ 有效数据太少，无法验证")
        return

    # 3. 统计误差
    t_list = np.array([T[:3, 3] for T in T_ET_list])
    t_mean = np.mean(t_list, axis=0)
    t_err = np.linalg.norm(t_list - t_mean, axis=1)
    rms_t = np.sqrt(np.mean(t_err**2))

    # 旋转误差（相对于平均旋转）
    R_list = [T[:3, :3] for T in T_ET_list]
    R_mean = R.from_matrix(R_list).mean().as_matrix()

    ang_err = []
    for R_i in R_list:
        dR = R_mean.T @ R_i
        ang = np.linalg.norm(R.from_matrix(dR).as_rotvec())
        ang_err.append(ang)

    rms_ang = np.sqrt(np.mean(np.array(ang_err)**2))

    # 4. 输出结果
    print("\n=== 验证结果 ===")
    print(f"End->Target 平移均值 (m): {t_mean}")
    print(f"平移 RMS 误差: {rms_t*1000:.3f} mm")
    print(f"旋转 RMS 误差: {np.degrees(rms_ang):.3f} deg")

    if rms_t < 0.002:
        print("✅ 标定质量：优秀")
    elif rms_t < 0.005:
        print("🟡 标定质量：可用")
    else:
        print("❌ 标定质量：较差，建议重新采集数据")

if __name__ == '__main__':
    main()

=== 手眼标定结果验证（眼在手外，棋盘在末端） ===
机器人数据: 25
视觉数据: 24

=== 验证结果 ===
End->Target 平移均值 (m): [ 0.03850464  0.04916085 -0.02194238]
平移 RMS 误差: 4.095 mm
旋转 RMS 误差: 0.907 deg
🟡 标定质量：可用


In [28]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

def visualize_plotly(T):
    fig = go.Figure()

    def add_frame(matrix, name, scale=0.2):
        origin = matrix[:3, 3]
        # 提取 X, Y, Z 轴向量
        axes = [matrix[:3, 0], matrix[:3, 1], matrix[:3, 2]]
        colors = ['red', 'green', 'blue']
        axis_names = ['X', 'Y', 'Z']

        for i in range(3):
            end_point = origin + axes[i] * scale
            fig.add_trace(go.Scatter3d(
                x=[origin[0], end_point[0]],
                y=[origin[1], end_point[1]],
                z=[origin[2], end_point[2]],
                mode='lines+text',
                line=dict(color=colors[i], width=5),
                text=["", f"{name}_{axis_names[i]}"],
                name=f"{name}_{axis_names[i]}"
            ))

    # 添加基座和相机
    add_frame(np.eye(4), "Base")
    add_frame(T, "Camera")

    fig.update_layout(scene=dict(aspectmode='data'))
    fig.show()
HAND_EYE_FILE = '/home/lgx/Project/hand_eye_calib/src/calibrate/result_hand_eye.txt'
T_cam_to_base = np.loadtxt(HAND_EYE_FILE)
# 将你的 4x4 矩阵传入即可
visualize_plotly(T_cam_to_base)